# **Klasifikasi Sentimen Menggunakan Recurrent Neural Network (RNN)**

## **Import Libraries**

In [1]:
# Import Libraries
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import gdown

## **Mengekstrak Data**

In [2]:
# gdown for downloading files directly from Google Drive.
from gdown import download

id = "1_uEtLA2GO2xjePWXOkjy_IlGM-2CdOWp"
output = "dataset_tweet_sentiment_opini_film.csv"
download(id=id, output=output)

Downloading...
From: https://drive.google.com/uc?id=1_uEtLA2GO2xjePWXOkjy_IlGM-2CdOWp
To: /content/dataset_tweet_sentiment_opini_film.csv
100%|██████████| 22.7k/22.7k [00:00<00:00, 14.3MB/s]


'dataset_tweet_sentiment_opini_film.csv'

In [3]:
df = pd.read_csv('dataset_tweet_sentiment_opini_film.csv')

In [4]:
df.head()

,Id,Sentiment,Text Tweet
0,1,negative,Jelek filmnya... apalagi si ernest gak mutu bg...
1,2,negative,Film king Arthur ini film paling jelek dari se...
2,3,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...
3,4,negative,Ane ga suka fast and furious..menurutku kok je...
4,5,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang..."


In [5]:
# Menghapus kolom yang tidak diperlukan
df = df.drop(columns=['Id'])

In [6]:
print("\n--- Data Awal (5 Baris Pertama) ---")
print(df.head())
print("\n")
print(df.info())


--- Data Awal (5 Baris Pertama) ---
  Sentiment                                         Text Tweet
0  negative  Jelek filmnya... apalagi si ernest gak mutu bg...
1  negative  Film king Arthur ini film paling jelek dari se...
2  negative  @beexkuanlin Sepanjang film gwa berkata kasar ...
3  negative  Ane ga suka fast and furious..menurutku kok je...
4  negative  @baekhyun36 kan gua ga tau film nya, lu bilang...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   200 non-null    object
 1   Text Tweet  200 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB
None


## **Preprocessing Data**

In [7]:
# --- Pra-pemrosesan Data ---

# 1. Pembersihan Teks Sederhana
def clean_text(text):
    text = text.lower() # Ubah ke huruf kecil
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) # Hapus mention (@user)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text) # Hapus URL
    text = re.sub(r'[^\w\s]', ' ', text) # Hapus tanda baca
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi berlebih
    return text

df['Text Tweet Cleaned'] = df['Text Tweet'].apply(clean_text)

print("\n--- Data Setelah Pembersihan Teks (5 Baris Pertama) ---")
print(df[['Text Tweet', 'Text Tweet Cleaned']].head())


--- Data Setelah Pembersihan Teks (5 Baris Pertama) ---
                                          Text Tweet  \
0  Jelek filmnya... apalagi si ernest gak mutu bg...   
1  Film king Arthur ini film paling jelek dari se...   
2  @beexkuanlin Sepanjang film gwa berkata kasar ...   
3  Ane ga suka fast and furious..menurutku kok je...   
4  @baekhyun36 kan gua ga tau film nya, lu bilang...   

                                  Text Tweet Cleaned  
0  jelek filmnya apalagi si ernest gak mutu bgt a...  
1  film king arthur ini film paling jelek dari se...  
2  sepanjang film gwa berkata kasar terus pada ba...  
3  ane ga suka fast and furious menurutku kok jel...  
4  kan gua ga tau film nya lu bilang perang peran...  


In [8]:
# Hitung rata-rata dan panjang maksimum tweet setelah pembersihan
tweet_lengths = df['Text Tweet Cleaned'].apply(lambda x: len(x.split()))
avg_len = tweet_lengths.mean()
max_len_calc = tweet_lengths.max()

print(f"\nRata-rata panjang tweet: {avg_len:.0f} kata")
print(f"Panjang tweet maksimum: {max_len_calc} kata")


Rata-rata panjang tweet: 15 kata
Panjang tweet maksimum: 25 kata


In [9]:
# Revisi MAX_LEN: Ambil nilai yang lebih realistis (misalnya rata-rata + 2 kali deviasi standar, atau maks 30)
MAX_LEN = min(int(avg_len + 2 * tweet_lengths.std()), 30)
if MAX_LEN < 15: MAX_LEN = 15 # Pastikan tidak terlalu pendek

print(f"MAX_LEN baru yang digunakan: {MAX_LEN}")

MAX_LEN baru yang digunakan: 25


In [10]:
le = LabelEncoder()
df['Sentiment_Encoded'] = le.fit_transform(df['Sentiment'])
# 'negative' -> 0, 'positive' -> 1 (berdasarkan abjad)

# Definisikan X (fitur) dan y (target)
X = df['Text Tweet Cleaned'].values
y = df['Sentiment_Encoded'].values

# Pemisahan Data Latih dan Uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

print(f"\nJumlah data latih: {len(X_train)}")
print(f"Jumlah data uji: {len(X_test)}")


Jumlah data latih: 180
Jumlah data uji: 20


In [11]:
# Parameter untuk Tokenizer dan Padding
VOCAB_SIZE = 7000 # Maksimal kata yang akan dipertahankan
OOV_TOKEN = "<OOV>" # Token untuk kata di luar vocabulary
MAX_LEN = 50

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(X_train)

# Mengubah teks menjadi urutan angka
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Menyamakan panjang semua sequence
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"\nBentuk data latih setelah padding: {X_train_padded.shape}")
print(f"Bentuk data uji setelah padding: {X_test_padded.shape}")


Bentuk data latih setelah padding: (180, 50)
Bentuk data uji setelah padding: (20, 50)


## **Modelling**

In [12]:
# --- Membangun Model SimpleRNN ---
vocab_size = len(tokenizer.word_index) + 1
EMBEDDING_DIM = 128

model = Sequential([
    # Layer Embedding untuk mengubah integer menjadi vektor padat
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN),

    # Layer Dropout
    Dropout(0.5),

    # Layer SimpleRNN
    SimpleRNN(16),

    # Layer output dengan aktivasi sigmoid untuk klasifikasi biner
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# --- Kompilasi Model ---
adam_optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# --- Melatih Model ---

# Konfigurasi EarlyStopping untuk memantau val_loss
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("Memulai proses training model...")
history = model.fit(X_train_padded, y_train,
                    epochs=50,
                    batch_size=16,
                    validation_data=(X_test_padded, y_test),
                    verbose=2,
                    callbacks=[early_stopping])

print("\nTraining selesai.")

Memulai proses training model...
Epoch 1/50
12/12 - 3s - 282ms/step - accuracy: 0.5000 - loss: 0.7761 - val_accuracy: 0.5000 - val_loss: 0.7402
Epoch 2/50
12/12 - 0s - 39ms/step - accuracy: 0.5111 - loss: 0.7091 - val_accuracy: 0.6500 - val_loss: 0.6811
Epoch 3/50
12/12 - 0s - 24ms/step - accuracy: 0.4778 - loss: 0.7033 - val_accuracy: 0.5000 - val_loss: 0.6814
Epoch 4/50
12/12 - 0s - 24ms/step - accuracy: 0.5833 - loss: 0.6796 - val_accuracy: 0.7000 - val_loss: 0.6774
Epoch 5/50
12/12 - 0s - 26ms/step - accuracy: 0.5389 - loss: 0.6893 - val_accuracy: 0.6500 - val_loss: 0.6769
Epoch 6/50
12/12 - 0s - 24ms/step - accuracy: 0.5444 - loss: 0.6800 - val_accuracy: 0.6500 - val_loss: 0.6760
Epoch 7/50
12/12 - 0s - 33ms/step - accuracy: 0.5444 - loss: 0.6769 - val_accuracy: 0.6500 - val_loss: 0.6751
Epoch 8/50
12/12 - 1s - 72ms/step - accuracy: 0.5611 - loss: 0.6738 - val_accuracy: 0.6500 - val_loss: 0.6732
Epoch 9/50
12/12 - 1s - 56ms/step - accuracy: 0.6000 - loss: 0.6697 - val_accuracy: 0.

## **Hasil dan Evaluasi**

In [15]:
# --- Evaluasi Model ---

print("\n--- Evaluasi Model pada Data Uji ---")
loss, accuracy = model.evaluate(X_test_padded, y_test, verbose=0)

print(f"Akurasi Model pada Data Uji: {accuracy*100:.2f}%")
print(f"Loss Model pada Data Uji: {loss:.4f}")


--- Evaluasi Model pada Data Uji ---
Akurasi Model pada Data Uji: 75.00%
Loss Model pada Data Uji: 0.6196


In [16]:
# --- Melakukan Prediksi pada Data Test ---

# Melakukan prediksi pada seluruh data test
predictions = model.predict(X_test_padded)
# Mengubah probabilitas menjadi kelas biner (0 atau 1)
predicted_classes = (predictions > 0.5).astype("int32")

# Membuat pemetaan dari indeks ke kata untuk decode sekuens
reverse_word_index = {value: key for key, value in tokenizer.word_index.items()}

def decode_sequence(sequence):
    # Fungsi untuk mengubah sekuens angka kembali menjadi teks
    return ' '.join([reverse_word_index.get(i, '?') for i in sequence if i != 0])

# Menampilkan 5 contoh hasil prediksi
print("\n" + "="*30)
print("Contoh Hasil Prediksi pada Data Test:")
print("="*30)

for i in range(5):
    print(f"{i+1}")

    # Mengambil teks asli dari sekuens yang sudah di-decode
    original_text = decode_sequence(X_test_padded[i])
    print(f"Teks Asli (decoded): {original_text}")

    # Label asli dan prediksi
    true_label = "Positif" if y_test[i] == 1 else "Negatif"
    predicted_label = "Positif" if predicted_classes[i][0] == 1 else "Negatif"

    print(f"Sentimen Asli    : {true_label}")
    print(f"Sentimen Prediksi : {predicted_label}")
    print(f"Probabilitas      : {predictions[i][0]:.4f}")
    print("-" * 20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step

Contoh Hasil Prediksi pada Data Test:
1
Teks Asli (decoded): <OOV> film bagian ini jijk parah
Sentimen Asli    : Negatif
Sentimen Prediksi : Negatif
Probabilitas      : 0.4272
--------------------
2
Teks Asli (decoded): <OOV> mbah film ini keren <OOV> to <OOV> <OOV> bisa <OOV> <OOV> abis nonton
Sentimen Asli    : Positif
Sentimen Prediksi : Negatif
Probabilitas      : 0.3215
--------------------
3
Teks Asli (decoded): <OOV> lebih <OOV> nilai film jelek <OOV> nilai film bagus <OOV>
Sentimen Asli    : Negatif
Sentimen Prediksi : Negatif
Probabilitas      : 0.4737
--------------------
4
Teks Asli (decoded): nah ini coba nonton filmnya deh asik parah
Sentimen Asli    : Positif
Sentimen Prediksi : Positif
Probabilitas      : 0.7176
--------------------
5
Teks Asli (decoded): ternyata <OOV> film <OOV> <OOV> jelekk
Sentimen Asli    : Negatif
Sentimen Prediksi : Negatif
Probabilitas      : 0.4467
--------------------


In [17]:
def predict_sentiment(text):
    """
    Fungsi untuk memprediksi sentimen dari sebuah kalimat input.
    """
    # 1. Bersihkan teks input menggunakan fungsi yang sama
    cleaned_text = clean_text(text)

    # 2. Ubah teks menjadi sekuens integer menggunakan tokenizer yang sama
    # Perlu diubah menjadi list karena tokenizer mengharapkan iterable
    sequence = tokenizer.texts_to_sequences([cleaned_text])

    # 3. Lakukan padding pada sekuens
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')

    # 4. Lakukan prediksi dengan model
    prediction = model.predict(padded_sequence)
    probability = prediction[0][0]

    # 5. Interpretasikan hasilnya
    print(f"Teks Input: '{text}'")
    if probability > 0.5:
        sentiment = "Positif"
        confidence = probability * 100
    else:
        sentiment = "Negatif"
        confidence = (1 - probability) * 100

    print(f"Prediksi Sentimen: {sentiment} (Keyakinan: {confidence:.2f}%)")
    print(f"Skor Probabilitas Mentah: {probability:.4f}")
    print("-" * 30)

# Contoh kalimat positif
kalimat_positif = "Filmnya keren banget, ceritanya seru dan aktingnya memukau!"
predict_sentiment(kalimat_positif)

# Contoh kalimat negatif
kalimat_negatif = "Ceritanya membosankan dan alurnya lambat sekali, saya kecewa."
predict_sentiment(kalimat_negatif)

# Coba dengan kalimat Anda sendiri
kalimat_custom = "Awalnya bagus tapi endingnya jelek banget"
predict_sentiment(kalimat_custom)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Teks Input: 'Filmnya keren banget, ceritanya seru dan aktingnya memukau!'
Prediksi Sentimen: Negatif (Keyakinan: 65.15%)
Skor Probabilitas Mentah: 0.3485
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Teks Input: 'Ceritanya membosankan dan alurnya lambat sekali, saya kecewa.'
Prediksi Sentimen: Positif (Keyakinan: 62.05%)
Skor Probabilitas Mentah: 0.6205
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Teks Input: 'Awalnya bagus tapi endingnya jelek banget'
Prediksi Sentimen: Positif (Keyakinan: 57.28%)
Skor Probabilitas Mentah: 0.5728
------------------------------


In [18]:
# --- OPSI TAMBAHAN: Prediksi Contoh Baru ---
def predict_sentiment(text_input):
    # Bersihkan teks
    cleaned_text = clean_text(text_input)
    # Tokenisasi
    seq = tokenizer.texts_to_sequences([cleaned_text])
    # Padding
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    # Prediksi
    prediction = model.predict(padded)[0][0]

    sentiment = "Positive" if prediction >= 0.5 else "Negative"

    return f"Teks: '{text_input}'\nPrediksi Probabilitas (Positive): {prediction:.4f}\nSentimen: {sentiment}"

# Contoh tweet
new_tweet_1 = "Film ini sangat bagus, saya suka plot twistnya!"
new_tweet_2 = "Jelek sekali, alur ceritanya membosankan dan bikin ngantuk."

print("\n--- Hasil Prediksi Contoh Baru ---")
print(predict_sentiment(new_tweet_1))
print("-" * 30)
print(predict_sentiment(new_tweet_2))


--- Hasil Prediksi Contoh Baru ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Teks: 'Film ini sangat bagus, saya suka plot twistnya!'
Prediksi Probabilitas (Positive): 0.6025
Sentimen: Positive
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Teks: 'Jelek sekali, alur ceritanya membosankan dan bikin ngantuk.'
Prediksi Probabilitas (Positive): 0.6404
Sentimen: Positive


In [19]:
# Tambahkan kode ini di Colab setelah model Anda selesai di-training

print("\n\n--- PENGUJIAN DENGAN DATA BARU (OUT-OF-SAMPLE) ---")

# Definisikan data baru dalam list
new_data_to_test = [
    "Akhirnya nonton! Film ini benar-benar karya masterpiece, sinematografinya memukau!", # Positif
    "Menyesal menghabiskan dua jam di bioskop, alurnya lambat dan akting para pemainnya datar.", # Negatif
    "Gila! Plot twist di akhir film bikin saya merinding. Sangat direkomendasikan.", # Positif
    "Ekspektasi tinggi, tapi eksekusi filmnya sangat mengecewakan. Cerita jadi berantakan.", # Negatif
    "Musik latarnya pas banget, sukses membawa emosi penonton. Lima bintang untuk film ini!", # Positif
    "Jujur, saya hampir tertidur di tengah film. Dialognya basi dan tidak ada ketegangan sama sekali.", # Negatif
    "Film horor terbaik tahun ini! Efek visualnya seram dan jalan ceritanya orisinal.", # Positif
    "Sayang sekali, humor yang disajikan dalam film komedi ini garing dan maksa.", # Negatif
    "Penampilan aktor utama sangat kuat dan mendalam. Layak dapat penghargaan.", # Positif
    "Dari awal sampai akhir, film ini terasa membingungkan. Jelas sekali ini film gagal." # Negatif
]

for i, tweet in enumerate(new_data_to_test):
    result = predict_sentiment(tweet)
    print(f"\n--- Data Uji #{i+1} ---")
    print(result)



--- PENGUJIAN DENGAN DATA BARU (OUT-OF-SAMPLE) ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

--- Data Uji #1 ---
Teks: 'Akhirnya nonton! Film ini benar-benar karya masterpiece, sinematografinya memukau!'
Prediksi Probabilitas (Positive): 0.4530
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

--- Data Uji #2 ---
Teks: 'Menyesal menghabiskan dua jam di bioskop, alurnya lambat dan akting para pemainnya datar.'
Prediksi Probabilitas (Positive): 0.6827
Sentimen: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

--- Data Uji #3 ---
Teks: 'Gila! Plot twist di akhir film bikin saya merinding. Sangat direkomendasikan.'
Prediksi Probabilitas (Positive): 0.3902
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

--- Data Uji #4 ---
Teks: 'Ekspektasi tinggi, tapi eksekusi filmnya sangat mengecewakan. Cerita jadi berantakan.'
Prediksi Probabilitas (Positive): 0.5264
Sentimen: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

--- Data Uji #5 ---
Teks: 'Musik latarnya pas banget, sukses me